In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets2/oof_preds_lgb.npy
/kaggle/input/datasets2/test_preds_nn_pytorch.npy
/kaggle/input/datasets2/test_preds_lgb.npy
/kaggle/input/datasets2/oof_preds_nn_pytorch.npy
/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv
/kaggle/input/firstlayer/oof_preds_hgb_fixed_params.npy
/kaggle/input/firstlayer/oof_preds_nn_keras.npy
/kaggle/input/firstlayer/oof_preds_nb_onehot.npy
/kaggle/input/firstlayer/oof_preds.npy
/kaggle/input/firstlayer/test_preds.npy
/kaggle/input/firstlayer/test_preds_nb_onehot.npy
/kaggle/input/firstlayer/test_preds_nn_keras.npy
/kaggle/input/firstlayer/test_preds_hgb_fixed_params.npy
/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


In [ ]:
import os, gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

pl.seed_everything(42)

def calculate_map3(y_true, y_pred):
    top3_preds_indices = np.argsort(y_pred, axis=1)[:, ::-1][:, :3]
    scores = []
    for i, true_label in enumerate(y_true):
        top3 = top3_preds_indices[i]
        score = 0.0
        if true_label in top3:
            rank = np.where(top3 == true_label)[0][0] + 1
            score = 1.0 / rank
        scores.append(score)
    return np.mean(scores)

class SpatialDropout1D(nn.Dropout2d):
    def forward(self, x):
        # x shape: (batch, channels, timesteps)
        x = x.unsqueeze(3)            # (batch, channels, timesteps, 1)
        x = super().forward(x)        # apply 2D dropout on (channels, timesteps)
        return x.squeeze(3)           # back to (batch, channels, timesteps)


# -----------------------------
# Dataset for Embedding
# -----------------------------
class TabularEmbeddingDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X.values, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long) if y is not None else None

    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        return (self.X[idx], self.y[idx]) if self.y is not None else self.X[idx]

# -----------------------------
# CNN Model with Embeddings
# -----------------------------
class SoftOrdering1DCNN(pl.LightningModule):
    def __init__(self, category_sizes, output_dim, embedding_size=16, lr=2e-4):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.loss_fn = nn.CrossEntropyLoss()
        self.val_outputs = []

        self.embeddings = nn.ModuleList([
            nn.Embedding(cat_size, embedding_size) for cat_size in category_sizes
        ])

        input_channels = len(category_sizes)
        self.cnn = nn.Sequential(
            nn.Conv1d(input_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            SpatialDropout1D(0.2),
            nn.AdaptiveAvgPool1d(2),
            nn.Flatten(),
            nn.Linear(128 * 2, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        embeds = [emb(x[:, i]) for i, emb in enumerate(self.embeddings)]
        x = torch.stack(embeds, dim=1)  # shape: [batch_size, channels, embed_dim]
        return self.cnn(x)

    def training_step(self, batch, _):
        x, y = batch
        return self.loss_fn(self(x), y)

    def validation_step(self, batch, _):
        x, y = batch
        logits = self(x)
        probs = torch.softmax(logits, dim=1).detach().cpu().numpy()
        y_true = y.cpu().numpy()
        self.val_outputs.append((y_true, probs))

    def on_validation_epoch_end(self):
        y_true = np.concatenate([out[0] for out in self.val_outputs])
        y_pred = np.concatenate([out[1] for out in self.val_outputs])
        map3 = calculate_map3(y_true, y_pred)
        self.log("val_map3", map3, prog_bar=True)
        self.val_outputs.clear()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

# -----------------------------
# Data Loading & Encoding
# -----------------------------
def load_data():
    train_synth = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv").drop(columns='id')
    train_orig = pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")
    test_df = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv").drop(columns='id')
    sub_df = pd.read_csv("/kaggle/input/playground-series-s5e6/sample_submission.csv")
    return train_synth, train_orig, test_df, sub_df

def prepare_categorical(df):
    for col in df.columns:
        df[col] = df[col].astype("category")
    return df

def encode_features(train_synth, train_orig, test_df):
    target_col = 'Fertilizer Name'
    label_enc = LabelEncoder()
    label_enc.fit(pd.concat([train_synth[target_col], train_orig[target_col]]))
    y_synth = label_enc.transform(train_synth[target_col])
    y_orig = label_enc.transform(train_orig[target_col])

    train_synth = train_synth.drop(columns=[target_col])
    train_orig = train_orig.drop(columns=[target_col])

    all_data = pd.concat([train_synth, train_orig, test_df], axis=0)
    all_data = prepare_categorical(all_data)

    for col in all_data.columns:
        all_data[col] = all_data[col].cat.codes

    n_synth = len(train_synth)
    n_orig = len(train_orig)

    X_synth = all_data.iloc[:n_synth]
    X_orig = all_data.iloc[n_synth:n_synth + n_orig]
    X_test = all_data.iloc[n_synth + n_orig:]

    category_sizes = [all_data[col].nunique() for col in all_data.columns]

    return X_synth, y_synth, X_orig, y_orig, X_test, label_enc, category_sizes

# -----------------------------
# Training Loop
# -----------------------------
def train_model():
    train_synth, train_orig, test_df_raw, sub_df = load_data()
    X_synth, y_synth, X_orig, y_orig, X_test, le, cat_sizes = encode_features(train_synth, train_orig, test_df_raw)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    test_preds = np.zeros((len(X_test), len(le.classes_)))
    oof_preds = np.zeros((len(X_synth), len(le.classes_)))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_synth, y_synth)):
        print(f"--- Fold {fold + 1}/5 ---")
        X_train, y_train = X_synth.iloc[train_idx], y_synth[train_idx]
        X_val, y_val = X_synth.iloc[val_idx], y_synth[val_idx]

        X_train_aug = pd.concat([X_train] + [X_orig] * 5, ignore_index=True)
        y_train_aug = np.concatenate([y_train] + [y_orig] * 5)

        train_ds = TabularEmbeddingDataset(X_train_aug, y_train_aug)
        val_ds = TabularEmbeddingDataset(X_val, y_val)
        test_ds = TabularEmbeddingDataset(X_test)

        train_loader = DataLoader(train_ds, batch_size=2048, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=2048)
        test_loader = DataLoader(test_ds, batch_size=2048)

        model = SoftOrdering1DCNN(category_sizes=cat_sizes, output_dim=len(le.classes_), lr=e-3)

        trainer = pl.Trainer(
            max_epochs=30,
            accelerator='gpu' if torch.cuda.is_available() else 'cpu',
            logger=False,
            enable_checkpointing=False,
            deterministic=True,
            log_every_n_steps=5
        )

        trainer.fit(model, train_loader, val_loader)

        model.eval()
        preds = []
        for batch in val_loader:
            x, _ = batch
            with torch.no_grad():
                preds.append(torch.softmax(model(x.to(model.device)), dim=1).cpu().numpy())
        oof_preds[val_idx] = np.concatenate(preds)

        preds = []
        for batch in test_loader:
            x = batch
            if isinstance(x, tuple): x = x[0]
            with torch.no_grad():
                preds.append(torch.softmax(model(x.to(model.device)), dim=1).cpu().numpy())
        test_preds += np.concatenate(preds) / 5
        gc.collect()

    print(f"Final MAP@3: {calculate_map3(y_synth, oof_preds):.5f}")

    top3_preds = np.argsort(test_preds, axis=1)[:, ::-1][:, :3]
    top3_labels = le.inverse_transform(top3_preds.flatten()).reshape(-1, 3)
    sub_df['Fertilizer Name'] = [' '.join(row) for row in top3_labels]
    sub_df.to_csv("submission_cnn_embedded.csv", index=False)
    print("Submission saved as 'submission_cnn_embedded.csv'")

# -----------------------------
if __name__ == "__main__":
    train_model()


--- Fold 1/5 ---


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

pl.seed_everything(42)

# -----------------------------
# MAP@3 scorer
# -----------------------------
def calculate_map3(y_true, y_pred):
    top3_preds_indices = np.argsort(y_pred, axis=1)[:, ::-1][:, :3]
    scores = []
    for i, true_label in enumerate(y_true):
        top3 = top3_preds_indices[i]
        score = 0.0
        if true_label in top3:
            rank = np.where(top3 == true_label)[0][0] + 1
            score = 1.0 / rank
        scores.append(score)
    return np.mean(scores)

# -----------------------------
# Dataset class
# -----------------------------
class FertilizerDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long) if y is not None else None

    def __len__(self): return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

# -----------------------------
# CNN model
# -----------------------------
class SoftOrdering1DCNN(pl.LightningModule):
    def __init__(self, input_dim, output_dim, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.loss_fn = nn.CrossEntropyLoss()
        self.lr = lr
        self.val_outputs = []

        self.model = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Unflatten(1, (16, 16)),         
            nn.Conv1d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=3, padding=1),  
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(2),           
            nn.Flatten(),                      
            nn.BatchNorm1d(256),
            nn.Linear(256, output_dim)
        )

    def forward(self, x): 
        return self.model(x)

    def training_step(self, batch, _):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        return loss

    def validation_step(self, batch, _):
        x, y = batch
        logits = self(x)
        probs = torch.softmax(logits, dim=1).detach().cpu().numpy()
        y_true = y.cpu().numpy()
        self.val_outputs.append((y_true, probs))

    def on_validation_epoch_end(self):
        y_true = np.concatenate([out[0] for out in self.val_outputs])
        y_pred = np.concatenate([out[1] for out in self.val_outputs])
        map3 = calculate_map3(y_true, y_pred)
        self.log("val_map3", map3, prog_bar=True)
        self.val_outputs.clear()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

# -----------------------------
# Load and preprocess data
# -----------------------------
def load_data():
    train_synth = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv").drop(columns='id')
    train_orig = pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")
    test_df = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv").drop(columns='id')
    sub_df = pd.read_csv("/kaggle/input/playground-series-s5e6/sample_submission.csv")
    return train_synth, train_orig, test_df, sub_df

def feature_eng(df):
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        df[f'{col}_Binned'] = pd.cut(df[col], bins=5, labels=False).astype('category')
    return df

def encode_features(train_synth, train_orig, test_df):
    target_col = 'Fertilizer Name'
    label_enc = LabelEncoder()
    label_enc.fit(pd.concat([train_synth[target_col], train_orig[target_col]]))
    y_synth = label_enc.transform(train_synth[target_col])
    y_orig = label_enc.transform(train_orig[target_col])

    train_synth = train_synth.drop(columns=[target_col])
    train_orig = train_orig.drop(columns=[target_col])

    full_df = pd.concat([train_synth, train_orig, test_df], axis=0)
    for col in full_df.columns:
        full_df[col] = full_df[col].astype(str).astype("category")

    full_encoded = pd.get_dummies(full_df)

    n_synth = len(train_synth)
    n_orig = len(train_orig)
    X_synth = full_encoded.iloc[:n_synth].copy()
    X_orig = full_encoded.iloc[n_synth:n_synth + n_orig].copy()
    X_test = full_encoded.iloc[n_synth + n_orig:].copy()

    return X_synth, y_synth, X_orig, y_orig, X_test, label_enc

# -----------------------------
# Main training loop
# -----------------------------
def train_model():
    train_synth, train_orig, test_df_raw, sub_df = load_data()

    train_synth = feature_eng(train_synth)
    train_orig = feature_eng(train_orig)
    test_df_raw = feature_eng(test_df_raw)

    X_synth, y_synth, X_orig, y_orig, X_test, le = encode_features(train_synth, train_orig, test_df_raw)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    test_preds = np.zeros((len(X_test), len(le.classes_)))
    oof_preds = np.zeros((len(X_synth), len(le.classes_)))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_synth, y_synth)):
        print(f"--- Fold {fold + 1}/5 ---")
        X_train, y_train = X_synth.iloc[train_idx], y_synth[train_idx]
        X_val, y_val = X_synth.iloc[val_idx], y_synth[val_idx]

        X_train_aug = pd.concat([X_train] + [X_orig] * 5, ignore_index=True)
        y_train_aug = np.concatenate([y_train] + [y_orig] * 5)

        train_ds = FertilizerDataset(X_train_aug, y_train_aug)
        val_ds = FertilizerDataset(X_val, y_val)
        test_ds = FertilizerDataset(X_test)

        train_loader = DataLoader(train_ds, batch_size=2048, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=2048)
        test_loader = DataLoader(test_ds, batch_size=2048)

        model = SoftOrdering1DCNN(input_dim=X_train.shape[1], output_dim=len(le.classes_),lr=1e-3)

        trainer = pl.Trainer(
            max_epochs=30,
            accelerator='gpu' if torch.cuda.is_available() else 'cpu',
            logger=False,
            enable_checkpointing=False,
            deterministic=True,
            log_every_n_steps=5
        )

        trainer.fit(model, train_loader, val_loader)

        model.eval()
        preds = []
        for batch in val_loader:
            x, _ = batch
            with torch.no_grad():
                preds.append(torch.softmax(model(x.to(model.device)), dim=1).cpu().numpy())
        oof_preds[val_idx] = np.concatenate(preds)

        preds = []
        for batch in test_loader:
            x = batch
            if isinstance(x, tuple): x = x[0]
            with torch.no_grad():
                preds.append(torch.softmax(model(x.to(model.device)), dim=1).cpu().numpy())
        test_preds += np.concatenate(preds) / 5

        gc.collect()

    print(f"Final MAP@3: {calculate_map3(y_synth, oof_preds):.5f}")

    top3_preds = np.argsort(test_preds, axis=1)[:, ::-1][:, :3]
    top3_labels = le.inverse_transform(top3_preds.flatten()).reshape(-1, 3)
    sub_df['Fertilizer Name'] = [' '.join(row) for row in top3_labels]
    sub_df.to_csv("submission_cnn_onehot.csv", index=False)
    print("Submission saved as 'submission_cnn_onehot.csv'")

# -----------------------------
if __name__ == "__main__":
    train_model()##can you do categorical enbedding and hot encoding

--- Fold 1/5 ---


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

--- Fold 3/5 ---


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import torch
import os
import gc

# Seed for reproducibility
np.random.seed(42)

def calculate_map3(y_true, y_pred_proba):
    top3_preds = np.argsort(y_pred_proba, axis=1)[:, ::-1][:, :3]
    score = 0.0
    for i in range(len(y_true)):
        if y_true[i] in top3_preds[i]:
            idx = np.where(top3_preds[i] == y_true[i])[0][0]
            score += 1.0 / (idx + 1)
    return score / len(y_true)

def load_data():
    train_synth = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv").drop(columns='id')
    train_orig = pd.read_csv("/kaggle/input/original/Fertilizer Prediction .csv")
    test_df = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv").drop(columns='id')
    sub_df = pd.read_csv("/kaggle/input/playground-series-s5e6/sample_submission.csv")
    return train_synth, train_orig, test_df, sub_df

def feature_eng(df):
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        df[f'{col}_Binned'] = pd.cut(df[col], bins=5, labels=False).astype('category')
    return df

def encode_features(train_synth, train_orig, test_df):
    target_col = 'Fertilizer Name'
    le_target = LabelEncoder()
    le_target.fit(pd.concat([train_synth[target_col], train_orig[target_col]]))
    y_synth = le_target.transform(train_synth[target_col])
    y_orig = le_target.transform(train_orig[target_col])

    categorical_cols = ['Soil Type', 'Crop Type'] + [col for col in train_synth.columns if '_Binned' in col]
    for col in categorical_cols:
        le_feat = LabelEncoder()
        full_col = pd.concat([train_synth[col].astype(str), train_orig[col].astype(str), test_df[col].astype(str)])
        le_feat.fit(full_col)
        train_synth[col] = le_feat.transform(train_synth[col].astype(str))
        train_orig[col] = le_feat.transform(train_orig[col].astype(str))
        test_df[col] = le_feat.transform(test_df[col].astype(str))

    return train_synth[categorical_cols], y_synth, train_orig[categorical_cols], y_orig, test_df[categorical_cols], le_target

def train_tabnet():
    train_synth, train_orig, test_df, sub_df = load_data()

    train_synth = feature_eng(train_synth)
    train_orig = feature_eng(train_orig)
    test_df = feature_eng(test_df)

    X_synth, y_synth, X_orig, y_orig, X_test, le = encode_features(train_synth, train_orig, test_df)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    test_preds = np.zeros((len(X_test), len(le.classes_)))
    oof_preds = np.zeros((len(X_synth), len(le.classes_)))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_synth, y_synth)):
        print(f"Fold {fold+1}/5")

        X_train, y_train = X_synth.iloc[train_idx], y_synth[train_idx]
        X_val, y_val = X_synth.iloc[val_idx], y_synth[val_idx]

        X_train_aug = pd.concat([X_train] + [X_orig] * 5, ignore_index=True)
        y_train_aug = np.concatenate([y_train] + [y_orig] * 5)

        clf = TabNetClassifier(
            seed=42,
            verbose=0,
            device_name='cuda' if torch.cuda.is_available() else 'cpu'
        )

        clf.fit(
            X_train_aug.values, y_train_aug,
            eval_set=[(X_val.values, y_val)],
            eval_metric=['accuracy'],
            max_epochs=200,
            patience=20,
            batch_size=1024,
            virtual_batch_size=128,
            drop_last=False
        )

        oof_preds[val_idx] = clf.predict_proba(X_val.values)
        test_preds += clf.predict_proba(X_test.values) / 5
        gc.collect()

    score = calculate_map3(y_synth, oof_preds)
    print(f"Final CV MAP@3: {score:.5f}")

    top3 = np.argsort(test_preds, axis=1)[:, ::-1][:, :3]
    top3_labels = le.inverse_transform(top3.flatten()).reshape(-1, 3)
    sub_df['Fertilizer Name'] = [' '.join(row) for row in top3_labels]
    sub_df.to_csv("submission_tabnet.csv", index=False)
    print("Saved submission_tabnet.csv")

train_tabnet()
